# Summarize results for Marek's thesis

This notebook is used to generate latex tables for Marek's thesis.

In [ ]:
import json
import os
import numpy as np
import pandas as pd

In [ ]:
def generate_full_vs_top_h_table(
    experiments,
    base_method = "optimal-instance-precision",
    table_header = "",
    multiplier = 100,
    per_page = 9,
    top_k = 5,
):
    basic_methods = {
        "optimal-instance-precision": "\\InfTopK",
        "optimal-instance-ps-precision": "\\InfPSK",
    }

    metrics = {
        "instance-precision": "Precision",
        "ps-precision": "Propensity-scored Prec.",
        "instance-recall": "Recall",
        "midrule": "\\midrule",
        "macro-precision": "Macro-Precision",
        "macro-recall": "Macro-Recall",
        #"macro-balanced-accuracy": "BA",
        "macro-f1": "Macro-F$_1$",
        #"macro-jaccard-score": "JS",
        # "macro-gmean": "GM",
        # "macro-hmean": "HM",
        "coverage": "Coverage"
    }

    table = ""
    for e, (experiment, experiment_label) in enumerate(experiments.items()):
        all_results = {}
        for top_k in [1, 3, 5]:
            filename = f"{experiment}/{base_method}_k={top_k}_v=0.0_s=13_results.json"
            if os.path.exists(filename):
                with open(filename, "r") as f:
                    result_file_data = json.load(f)
                for metric, metric_label in metrics.items():
                    metric = f"{metric}@{top_k}"
                    if metric in result_file_data:
                        all_results[f"{metric}_all"] = result_file_data[metric] * multiplier

            filename = filename.replace("_k=", "_top_labels=0.2_k=")
            filename = filename.replace("org2", "org4")
            if os.path.exists(filename):
                with open(filename, "r") as f:
                    result_file_data = json.load(f)
                for metric, metric_label in metrics.items():
                    metric = f"{metric}@{top_k}"
                    if metric in result_file_data:
                        all_results[f"{metric}_top"] = result_file_data[metric] * multiplier
        
        print(all_results)
        results = []
        for metric, metric_label in metrics.items():
            _result = {"Metric": metric_label}
            for top_k in [1, 3, 5]:
                _metric = f"{metric}@{top_k}"
                _result[f"{top_k}_all"] = all_results.get(f"{_metric}_all", "-")
            for top_k in [1, 3, 5]:
                _metric = f"{metric}@{top_k}"
                _ref_value = all_results.get(f"{_metric}_all", "-")
                _value = all_results.get(f"{_metric}_top", "-")
                _diff = None
                if _ref_value != "-" and _value != "-":
                    _diff = (_value - _ref_value) / _ref_value * 100
                if _value != "-":
                    _value = f"{_value:.2f}"
                if _diff is not None:
                    color = r"\color{green}"
                    if _diff < -5:
                        color = r"\color{orange}"
                    if _diff < -30:
                        color = r"\color{red}"
                    _diff = f" {{\\scriptsize {color} ({_diff:.2f}\\%)}}"
                _result[f"{top_k}_top"] = _value
                _result[f"{top_k}_top_diff"] = _diff
                
                
            results.append(_result)

        df = pd.DataFrame(results)
        
        table += "\n"
        if e % per_page == 0:
            if e == 0:
                table += "\\begin{table}\n"
                table += table_header
                table += "\\small\n"
            table += """
\\centering
\\resizebox{\\linewidth}{!}{
\\begin{tabular}{l|rrr|rlrlrl}
\\toprule
    Metric & \\multicolumn{3}{c|}{With all labels} & \\multicolumn{6}{c}{With top 20\\% labels} \\\\
    & \\multicolumn{1}{c}{$@1$} & \\multicolumn{1}{c}{$@3$} & \\multicolumn{1}{c|}{$@5$} 
    & \\multicolumn{2}{c}{$@1$} & \\multicolumn{2}{c}{$@3$} & \\multicolumn{2}{c}{$@5$} \\\\
"""
# Metric & \\multicolumn{3}{c|}{Class. with all labels} & \\multicolumn{6}{c}{Classifier with top 20\\% labels} \\\\
        table += f"\\midrule\n"
        table += f"\\multicolumn{{10}}{{c}}{{{experiment_label}}} \\\\\n"

        # Print the results as a latex table
        latex_table = df.to_latex(index=False, float_format="{:0.2f}".format)
        #latex_table = df.to_latex(index=False)
        table += "\n    ".join(latex_table.split("\n")[3:-3]).replace("\midrule & - & - & - & - & NaN & - & NaN & - & NaN \\\\", "\midrule")

        if e % per_page == per_page - 1 or e == len(experiments) - 1:
            table += """
\\bottomrule
\\end{tabular}
}
\\end{table}
"""

    return table

In [ ]:
org_exp_dir = "results_thesis_org2"

experiments = {
    f"../{org_exp_dir}/rcv1x_100_plt": "RCV1x-2K",
    f"../{org_exp_dir}/eurlex_100_plt": "Eurlex-4K",
    f"../{org_exp_dir}/EURLex-4.3K_100_plt": "Eurlex-4.3K",
    f"../{org_exp_dir}/amazonCat_100_plt": "AmazonCat-13K",
    f"../{org_exp_dir}/amazonCat-14K_100_plt": "AmazonCat-14K",
    #f"../{org_exp_dir}/wiki10_100_plt": "Wiki10-31K",
    #f"../{org_exp_dir}/deliciousLarge_100_plt": "DeliciousLarge-200K",
    f"../{org_exp_dir}/wikiLSHTC_100_plt": "WikiLSHTC-325K",
    f"../{org_exp_dir}/WikipediaLarge-500K_100_plt": "WikipediaLarge-500K",
    f"../{org_exp_dir}/amazon_100_plt": "Amazon-670K",
}

header_main = r"""
\caption{Results (\%) of a classifier trained on the full set of labels
and a classifier trained with only the top 20\% of labels (most frequent labels) 
on different metrics budgeted at $k$ ($@k$).
}
\label{tab:all-vs-top-h}
"""

with open("tables_thesis/results-all-vs-top-h.tex", "w") as f:
    f.write(generate_full_vs_top_h_table(experiments, table_header=header_main, per_page=9))

## Main results

In [ ]:


EPSS = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 4e-5, 5e-5, 6e-5, 9e-5, 1e-4, 2e-4, 3e-4, 4e-4, 5e-4, 6e-4, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2, 3e-5, 5e-5, 1e-4, 3e-4, 5e-4, 1e-3, 3e-3, 5e-3, 1e-2]
#EPSS = [1e-8, 1e-6, 1e-4]
TOL = 1e-7
seeds = [13, 1988, 1993, 2023, 2024]
seeds = [13, 1988, 1993, 2023, 2024]
val_split = 0.0

metrics = {
    "instance-precision": "P",
    "ps-precision": "PS",
    "instance-recall": "R",
    "macro-precision": "P",
    "macro-recall": "R",
    "macro-balanced-accuracy": "BA",
    "macro-f1": "F",
    "macro-jaccard-score": "JS",
    # "macro-gmean": "GM",
    # "macro-hmean": "HM",
    "coverage": "C"
}


def generate_table_with_results(
    experiments,
    table_header = "",
    multiplier = 100,
    per_page = 3,
    add_std = False,
    top_k = 5,
):
    basic_methods = {
        "optimal-instance-precision": "\\InfTopK",
        "optimal-instance-ps-precision": "\\InfPSK",
        "power-law-with-beta=0.25": "\\InfPowerK$_{\\beta=0.25}$",
        "power-law-with-beta=0.5": "\\InfPowerK$_{\\beta=0.5}$",
        #"power-law-with-beta=0.75": "\\InfPowerK",
        "log": "\\InfLogK",
        "optimal-macro-recall": "\\InfMacR",
        "optimal-macro-balanced-accuracy": "\\InfMacBA",
        "midrule1": "\\midrule",
    }
    methods = basic_methods.copy()
    for eps in EPSS:
        methods.update({
            f"block-coord-macro-precision-tol={TOL}-eps={eps}": f"\\InfBCAMacP$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-recall-tol={TOL}-eps={eps}": f"\\InfBCAMacR$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-balanced-accuracy-tol={TOL}-eps={eps}": f"\\InfBCAMacBA$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-f1-tol={TOL}-eps={eps}": f"\\InfBCAMacF$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-jaccard-score-tol={TOL}-eps={eps}": f"\\InfBCAMacJS$_{{\\epsilon={eps}}}$",
        })

    methods.update({
        f"block-coord-coverage-tol={TOL}": "\\InfBCACov",
        "midrule2": "\\midrule",
    })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-precision-eps={eps}": f"\\InfFWMacP$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-recall-eps={eps}": f"\\InfFWMacR$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-balanced-accuracy-eps={eps}": f"\\InfFWMacBA$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-f1-eps={eps}": f"\\InfFWMacF$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-jaccard-score-eps={eps}": f"\\InfFWMacJS$_{{\\epsilon={eps}}}$",
        })

    # methods.update({
    #     "midrule3": "\\midrule",
    # })

    # thr = 0.05
    # methods.update({
    #     "optimal-instance-precision_proba_threshold={thr}": "\\InfTopK",
    #     "optimal-instance-ps-precision_proba_threshold={thr}": "\\InfPSK",
    #     "power-law-with-beta=0.25_proba_threshold={thr}": "\\InfPowerK$_{\\beta=0.25}$",
    #     "power-law-with-beta=0.5_proba_threshold={thr}": "\\InfPowerK$_{\\beta=0.5}$",
    #     #"power-law-with-beta=0.75": "\\InfPowerK",
    #     "log_proba_threshold={thr}": "\\InfLogK",
    #     "optimal-macro-recall_proba_threshold={thr}": "\\InfMacR",
    #     "optimal-macro-balanced-accuracy_proba_threshold={thr}": "\\InfMacBA",
    #     "midrule1": "\\midrule",
    # })

    # for eps in EPSS:
    #     methods.update({
    #         f"frank-wolfe-macro-precision-eps={eps}_proba_threshold={thr}": f"\\InfFWMacP$_{{\\epsilon={eps}}}$",
    #     })

    # for eps in EPSS:
    #     methods.update({
    #         f"frank-wolfe-macro-recall-eps={eps}_proba_threshold={thr}": f"\\InfFWMacR$_{{\\epsilon={eps}}}$",
    #     })

    # for eps in EPSS:
    #     methods.update({
    #         f"frank-wolfe-macro-balanced-accuracy-eps={eps}_proba_threshold={thr}": f"\\InfFWMacBA$_{{\\epsilon={eps}}}$",
    #     })

    # for eps in EPSS:
    #     methods.update({
    #         f"frank-wolfe-macro-f1-eps={eps}_proba_threshold={thr}": f"\\InfFWMacF$_{{\\epsilon={eps}}}$",
    #     })

    # for eps in EPSS:
    #     methods.update({
    #         f"frank-wolfe-macro-jaccard-score-eps={eps}_proba_threshold={thr}": f"\\InfFWMacJS$_{{\\epsilon={eps}}}$",
    #     })


    # for eps in EPSS:
    #     methods.update({
    #         f"block-coord-macro-gmean-tol={TOL}-eps={eps}": f"\\InfBCAMacGM$_{{\\epsilon={eps}}}$",
    #         f"frank-wolfe-macro-gmean-eps={eps}": f"\\InfFWMacGM$_{{\\epsilon={eps}}}$",
    #     })

    # for eps in EPSS:
    #     methods.update({
    #         f"block-coord-macro-hmean-tol={TOL}-eps={eps}": f"\\InfBCAMacHM$_{{\\epsilon={eps}}}$",
    #         f"frank-wolfe-macro-hmean-eps={eps}": f"\\InfFWMacHM$_{{\\epsilon={eps}}}$",
    #     })



    formats = ["\\textbf{{{}}}", "\\textit{{{}}}"]

    table = ""
    for e, (experiment, experiment_label) in enumerate(experiments.items()):
        results = []
        prev_method = ""
        for method, method_label in methods.items():
            method_results = {
                "_method": method,
                "method": method_label
            }
            if "midrule" in method:
                results.append(method_results)
                continue

            for seed in seeds:
                filename = f"{experiment}/{method}_sample_test_labels_k={top_k}_v={val_split}_s=13_sample_s={seed}_results.json"
                if not os.path.exists(filename):
                    filename = f"{experiment}/{method}_k={top_k}_v={val_split}_s={seed}_results.json"

                if not os.path.exists(filename):
                    filename = filename.replace("org5", "org3")
                
                if not os.path.exists(filename):
                    filename = filename.replace("org3", "org2")
                    
                if os.path.exists(filename):
                    with open(filename, "r") as f:
                        result_file_data = json.load(f)
                    for metric, metric_label in metrics.items():
                        metric = f"{metric}@{top_k}"
                        if metric in result_file_data:
                            method_results.setdefault(metric, []).append(result_file_data[metric] * multiplier)
                else:
                    #print(f"File {filename} not found")
                    pass

            for k, v in list(method_results.items()):
                if isinstance(v, list) and isinstance(v[0], float):
                    method_results[k] = np.mean(v)
                    if add_std:
                        std = np.std(v)
                        if method in basic_methods and std == 0:
                           continue 
                        method_results[f"{k}_std"] = np.std(v)

            for metric in metrics.keys():
                if metric in method and metric in prev_method and method.split("-")[0] == prev_method.split("-")[0]:
                    metric_at_k = f"{metric}@{top_k}"
                    new_val = method_results.get(metric_at_k, 0)
                    prev_val = results[-1].get(metric_at_k, 0)
                    if new_val > prev_val and isinstance(new_val, float):
                        results[-1] = method_results
                    break
            else:
                results.append(method_results)
            prev_method = method

        for r in results:
            for k, v in r.items():
                if isinstance(v, float):
                    r[k] = np.round(v, 2)

        # Specialized method worst results
        specialized_method_worst_results = {}
        bca_method_results = {}
        fw_method_results = {}

        for i, r in enumerate(results):
            method = r["_method"]
            for metric in metrics.keys():
                if "_std" in metric:
                    continue

                metric_at_k = f"{metric}@{top_k}"
                if (metric in method or ("instance-recall" in metric and method == "optimal-instance-precision")) and metric_at_k in r:
                    if isinstance(r[metric_at_k], float):
                        specialized_method_worst_results[metric_at_k] = min(specialized_method_worst_results.get(metric_at_k,1000), r[metric_at_k])
                    if "block" in method:
                        bca_method_results[metric_at_k] = min(bca_method_results.get(metric_at_k,1000), r[metric_at_k])
                    if "frank" in method:
                        fw_method_results[metric_at_k] = min(fw_method_results.get(metric_at_k,1000), r[metric_at_k])

        # print(specialized_method_worst_results)
        # print(bca_method_results)
        # print(fw_method_results)

        # Select best in column
        for metric in list(metrics.keys()):
            if "_std" in metric:
                continue
            
            metric_at_k = f"{metric}@{top_k}"
            column = np.array([result.get(metric_at_k, 0) for result in results])
            argsort = np.flip(np.argsort(column))
            vals = column[argsort]
            for result in results:
                if metric_at_k not in result:
                    continue

                if add_std and metric_at_k + "_std" in result:
                    formated_result = f"\\makecell{{\\linespread{{1.0}} {result[metric_at_k]:.2f} \\\\ {{\\scriptsize $\\pm$ {result[metric_at_k+'_std']:.2f}}}}}"
                    del result[metric_at_k + "_std"]
                else:
                    formated_result = f"{result[metric_at_k]:.2f}"

                if result[metric_at_k] < specialized_method_worst_results.get(metric_at_k, 0):
                    method = r["_method"]
                    result[metric_at_k] = f"{{\\color{{gray!75}} {formated_result}}}"
                else:
                    result[metric_at_k] = formated_result
                    
                    # if "block" in method and (result[metric_at_k] < bca_method_results.get(metric_at_k,0) or result[metric_at_k] < specialized_method_worst_results.get(metric_at_k,0)):
                    #     result[metric_at_k] = f"{{\\color{{gray!75}} {result[metric_at_k]:.2f}}}"
                    # elif "frank" in method and result[metric_at_k] < fw_method_results.get(metric_at_k,0) or result[metric_at_k] < specialized_method_worst_results.get(metric_at_k,0)):
                    #     result[metric_at_k] = f"{{\\color{{gray!75}} {result[metric_at_k]:.2f}}}"
                    # elif result[metric_at_k] < specialized_method_worst_results.get(metric_at_k,0):
                    #     result[metric_at_k] = f"{{\\color{{gray!75}} {result[metric_at_k]:.2f}}}"
            
            f = -1
            prev_val = -1
            for idx, val in zip(argsort, vals):
                if prev_val != val:
                    f += 1
                    if f == len(formats):
                        break
                results[idx][metric_at_k] = formats[f].format(results[idx][metric_at_k])
                prev_val = val

            # for format, idx in zip(formats, argsort):
            #     results[idx][metric_at_k] = format.format(column[idx])

        color = "green!25"
        # Color columns with target
        for i, r in enumerate(results):
            method = r["_method"]
            del r["_method"]
            if "epsilon" in r["method"]:
                r["method"] = r["method"].split("$")[0]
            for metric in metrics.keys():
                metric_at_k = f"{metric}@{top_k}"
                if (metric in method or ("instance-recall" in metric and method == "optimal-instance-precision")) and metric_at_k in results[i]:
                    mark = ""
                    if "instance-recall" in metric:
                        if "sys" not in experiment:
                            color = "blue!25"
                        else:
                            color = "blue!12"
                    else:
                        color = "green!25"
                    if "instance-recall" in metric:
                        mark = "*"
                    if isinstance(results[i][metric_at_k], str):
                        results[i][metric_at_k] = f"\\cellcolor{{{color}}}{mark} {results[i][metric_at_k]}"
                    else:
                        results[i][metric_at_k] = f"\\cellcolor{{{color}}}{mark} {results[i][metric_at_k]:.2f}"
        

        df = pd.DataFrame(results)
        
        table += "\n"
        if e % per_page == 0:
            table += "\\begin{table}\n"
            if e == 0:
                table += table_header
            if add_std:
                table += "\\scriptsize\n"
            else:
                table += "\\small\n"
            table += """
\\centering
\\resizebox{\\linewidth}{!}{
\\begin{tabular}{l|rrr|rrrrrr}
\\toprule
    Method & \\multicolumn{3}{c|}{Instance $@__K__$} & \\multicolumn{6}{c}{Macro $@__K__$} \\\\
    & \\multicolumn{1}{c}{P} & \\multicolumn{1}{c}{PS} & \\multicolumn{1}{c|}{R} 
    & \\multicolumn{1}{c}{P} & \\multicolumn{1}{c}{R} & \\multicolumn{1}{c}{BA} & 
    \\multicolumn{1}{c}{F$_1$} & \\multicolumn{1}{c}{JS} & \\multicolumn{1}{c}{Cov} \\\\
""".replace("__K__", str(top_k))

        table += f"\\midrule\n"
        table += f"\\multicolumn{{10}}{{c}}{{{experiment_label}}} \\\\\n"

        # Print the results as a latex table
        latex_table = df.to_latex(index=False, float_format="{:0.2f}".format)
        #latex_table = df.to_latex(index=False)
        table += "\n    ".join(latex_table.split("\n")[3:-3]).replace("\midrule & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN & NaN \\\\", "\midrule")

        if e % per_page == per_page - 1 or e == len(experiments) - 1:
            table += """
\\bottomrule
\\end{tabular}
}
\\end{table}
"""

    return table

In [ ]:
os.makedirs("tables_thesis", exist_ok=True)

syn_exp_dir = "results_thesis_syn4"
org_exp_dir = "results_thesis_org3"

experiments = {
    f"../{org_exp_dir}/rcv1x_100_plt": "RCV1x-2K",
    f"../{org_exp_dir}/eurlex_100_plt": "Eurlex-4K",
    f"../{org_exp_dir}/EURLex-4.3K_100_plt": "Eurlex-4.3K",
    f"../{org_exp_dir}/amazonCat_100_plt": "AmazonCat-13K",
    f"../{org_exp_dir}/amazonCat-14K_100_plt": "AmazonCat-14K",
    f"../{org_exp_dir}/wiki10_100_plt": "Wiki10-31K",
    #f"../{org_exp_dir}/deliciousLarge_100_plt": "DeliciousLarge-200K",
    f"../{org_exp_dir}/wikiLSHTC_100_plt": "WikiLSHTC-325K",
    f"../{org_exp_dir}/WikipediaLarge-500K_100_plt": "WikipediaLarge-500K",
    f"../{org_exp_dir}/amazon_100_plt": "Amazon-670K",
}

header_main = r"""
\caption{Results (\%) on \emph{original} XMLC datasets with marginal conditional probabilities coming from PLT model for $k = 5$.
The \smash{\colorbox{green!25}{green background}} indicates cells in which the inference algorithm matches the metric it optimizes. 
The {\color{gray!75} gray text} indicate results below all the results with \smash{\colorbox{green!25}{green background}} for the given metric. 
The best results are in \textbf{bold}, and the second best are in \textit{italic}.
* -- while \InfTopK{} in general is not optimal for \RecallAtK{}, we expected it to be the closest to the optimal solution, and we mark \smash{\colorbox{blue!12}{blue background}}
}
\label{tab:main-plt-results}
"""

with open("tables_thesis/results-main-plt.tex", "w") as f:
    f.write(generate_table_with_results(experiments, table_header=header_main, per_page=3, add_std=False, top_k=5))

header_app = r"""
\caption{Results (\%) on \emph{original} XMLC datasets with marginal conditional probabilities coming from PLT model for $k \in \{1, 3, 5\}$.
The \smash{\colorbox{green!25}{green background}} indicates cells in which the inference algorithm matches the metric it optimizes. 
The {\color{gray!75} gray text} indicate results below all the results with \smash{\colorbox{green!25}{green background}} for the given metric. 
The best results are in \textbf{bold}, and the second best are in \textit{italic}.
* -- while \InfTopK{} in general is not optimal for \RecallAtK{}, we expected it to be the closest to the optimal solution, and we mark \smash{\colorbox{blue!12}{blue background}}.
}
\label{tab:app-plt-results}
"""

with open("tables_thesis/results-app-plt.tex", "w") as f:
    table = generate_table_with_results(experiments, table_header=header_app, per_page=2, add_std=True, top_k=1)
    table += generate_table_with_results(experiments, per_page=2, add_std=True, top_k=3)
    table += generate_table_with_results(experiments, per_page=2, add_std=True, top_k=5)
    f.write(table)


experiments = {
    f"../{syn_exp_dir}/rcv1x_100_plt": "Synthetic RCV1x-2K",
    f"../{syn_exp_dir}/eurlex_100_plt": "Synthetic Eurlex-4K",
    f"../{syn_exp_dir}/EURLex-4.3K_100_plt": "Synthetic Eurlex-4.3K",
    f"../{syn_exp_dir}/amazonCat_100_plt": "Synthetic AmazonCat-13K",
    f"../{syn_exp_dir}/amazonCat-14K_100_plt": "Synthetic AmazonCat-14K",
    f"../{syn_exp_dir}/wiki10_100_plt": "Synthetic Wiki10-31K",
    f"../{syn_exp_dir}/wikiLSHTC_100_plt": "Synthetic WikiLSHTC-325K",
    f"../{syn_exp_dir}/WikipediaLarge-500K_100_plt": "Synthetic WikipediaLarge-500K",
    f"../{syn_exp_dir}/amazon_100_plt": "Synthetic Amazon-670K",
}

header_main = r"""
\caption{Results (\%)  on \emph{synthetic versions} of XMLC datasets with ideal estimates of marginal conditional probabilities $\Marginals(\Instance) = \PredMarginals(\Instance)$ for $k = 5$. 
The \smash{\colorbox{green!25}{green background}} indicates cells in which the inference algorithm matches the metric it optimizes. 
The {\color{gray!75} gray text} indicate results below all the results with \smash{\colorbox{green!25}{green background}} for the given metric. 
The best results are in \textbf{bold}, and the second best are in \textit{italic}.
* -- because in this experiment we sample labels independently, \InfTopK{} becomes optimal strategy for \RecallAtK{} as showed in \cref{thm:prec-recall-equ-with-independence}.
}
\label{tab:main-plt-syn-results}
"""

with open("tables_thesis/results-main-plt-syn.tex", "w") as f:
    f.write(generate_table_with_results(experiments, table_header=header_main, per_page=3, add_std=False, top_k=5))

header_app = r"""
\caption{Results (\%) on \emph{synthetic versions} of XMLC datasets with ideal estimates of marginal conditional probabilities $\Marginals(\Instance) = \PredMarginals(\Instance)$ for $k \in \{1, 3, 5\}$.
The \smash{\colorbox{green!25}{green background}} indicates cells in which the inference algorithm matches the metric it optimizes. 
The {\color{gray!75} gray text} indicate results below all the results with \smash{\colorbox{green!25}{green background}} for the given metric. 
The best results are in \textbf{bold}, and the second best are in \textit{italic}.
* -- because in this experiment we sample labels independently, \InfTopK{} becomes optimal strategy for \RecallAtK{} as showed in \cref{thm:prec-recall-equ-with-independence}.
}
\label{tab:app-plt-syn-results}
"""

with open("tables_thesis/results-app-plt-syn.tex", "w") as f:
    table = generate_table_with_results(experiments, table_header=header_app, per_page=2, add_std=True, top_k=1)
    table += generate_table_with_results(experiments, per_page=2, add_std=True, top_k=3)
    table += generate_table_with_results(experiments, per_page=2, add_std=True, top_k=5)
    f.write(table)


In [ ]:
# Function for creating the plots

import json
import matplotlib.pyplot as plt
import numpy as np
from adjustText import adjust_text
from collections.abc import Iterable
import os

top_k = 5
multiplier = 100

plt.rcParams.update({
    "figure.figsize": (4, 2), # for smaller plots
    #"figure.figsize": (4, 2.5), # ICLR paper
    #"figure.figsize": (4, 3), # NeurIPS paper
    "figure.dpi": 300,
    "figure.autolayout": False,
    "text.usetex": True,
    'mathtext.fontset': 'stix',
    'font.family': 'STIXGeneral',
    'savefig.transparent': False,
})

plt.rcParams["text.latex.preamble"] = r"""
\usepackage[T1]{fontenc}
\usepackage{bold-extra}
\usepackage{amsmath}
\usepackage{amsfonts}
\usepackage{amssymb}
\newcommand{\InfTopK}{Top-K}
\newcommand{\InfPSK}{PS-K}
\newcommand{\InfPowerK}{Pow-K}
\newcommand{\InfLogK}{Log-K}
\newcommand{\InfMacR}{Macro-R$_{\text{prior}}$}
\newcommand{\InfMacBA}{Macro-BA$_{\text{prior}}$}
\newcommand{\InfBCA}[1]{BCA(#1)}
\newcommand{\InfBCAMacP}{\InfBCA{Macro-P}}
\newcommand{\InfBCAMacR}{\InfBCA{Macro-R}}
\newcommand{\InfBCAMacF}{\InfBCA{Macro-F1}}
\newcommand{\InfBCAMacBA}{\InfBCA{Macro-BA}}
\newcommand{\InfBCAMacJ}{\InfBCA{Macro-J}}
\newcommand{\InfBCAMacGM}{\InfBCA{Macro-G-M}}
\newcommand{\InfBCAMacHM}{\InfBCA{Macro-H-M}}
\newcommand{\InfBCACov}{\InfBCA{Cov}}
\newcommand{\InfFW}[1]{FW(#1)}
\newcommand{\InfFWMacP}{\InfFW{Macro-P}}
\newcommand{\InfFWMacR}{\InfFW{Macro-R}}
\newcommand{\InfFWMacF}{\InfFW{Macro-F1}}
\newcommand{\InfFWMacBA}{\InfFW{Macro-BA}}
\newcommand{\InfFWMacJ}{\InfFW{Macro-J}}
\newcommand{\InfFWMacGM}{\InfFW{Macro-G-M}}
\newcommand{\InfFWMacHM}{\InfFW{Macro-H-M}}
"""


def load_json(filepath):
    with open(filepath) as file:
        return json.load(file)


def plot_results(experiment, results, methods, x_axis, y_axis,
                 x_axis_label=None, y_axis_label=None, title=None, legend=False, add_std=False, on_plot_labels=False):
    all_labels = []
    x_rep = []
    y_rep = []

    fig = plt.figure()
    fig.patch.set_alpha(0)

    plt.clf()
    ax = plt.gca()
    ax.set_facecolor('white')

    for n, v in methods.items():
        x_vals = []
        y_vals = []
        x_errors = []
        y_errors = []
        labels = []
        if isinstance(v, str):
            try:
                labels.append(v)
                x_vals.append(results[n][x_axis])
                y_vals.append(results[n][y_axis])
                x_errors.append(results[n][f"{x_axis}_std"])
                y_errors.append(results[n][f"{y_axis}_std"])
            except KeyError as e:
                print(f"KeyError: {n} {x_axis} {y_axis}")
                print(f"Available keys: {results[n].keys()}")
                raise e
        elif isinstance(v, dict):
            for n2, v2 in v.items():
                try:
                    labels.append(v2)
                    x_vals.append(results[n2][x_axis])
                    y_vals.append(results[n2][y_axis])
                    x_errors.append(results[n2][f"{x_axis}_std"])
                    y_errors.append(results[n2][f"{y_axis}_std"])
                except KeyError as e:
                    print(f"KeyError: {n2} {x_axis} {y_axis}")
                    print(f"Available keys: {results[n2].keys()}")
                    raise e

        if on_plot_labels:
            all_labels.extend([plt.text(x, y, l, size=8) for x, y, l in zip(x_vals, y_vals, labels) if len(l)])
        x_rep.extend(x_vals)
        y_rep.extend(y_vals)
        plot_kwargs = {}
        if isinstance(v, str):
            plot_kwargs["label"] = v
        else:
            plot_kwargs["linestyle"] = "-"
            plot_kwargs["label"] = labels[0] #+ " - " + labels[-1]
        
        x_vals = np.array(x_vals)
        y_vals = np.array(y_vals)
        x_errors = np.array(x_errors)
        y_errors = np.array(y_errors)

        plt.plot(x_vals, y_vals, '.', **plot_kwargs)
        if add_std:
            plt.fill_between(x=x_vals, y1 = y_vals - y_errors, y2= y_vals + y_errors, alpha=0.3)
        #plt.errorbar(x_vals, y_vals, xerr=x_errors, yerr=y_errors, fmt='.', linestyle="-", linewidth=1, capsize=2, capthick=1)

    if on_plot_labels:
        adjust_text(all_labels, x_rep, y_rep, 
                    min_arrow_len=50,
                    #force_text=(0.2, 0.5),
                    #force_static=(0.2, 0.5),
                    #force_explode=(0.2, 0.5),
                    #expand=(1.4, 1.6),
                    time_lim=1, 
                    explode_radius=100,
                    arrowprops={"arrowstyle": "->", "lw": 0.5},
                    expand=(1.4, 1.5),
                    only_move='y-', #Only allow movement to the left
                    #only_move = {"text": "y", "static": "y", "explode": "y", "pull": "y"},
                    )
    
    if "syn" in experiment:
        experiment = experiment.split("/")[-1] + '_sys'
        #title = "Synthetic " + title
    else:
        experiment = experiment.split("/")[-1] + '_org'

    if legend:
        plt.legend(loc=3, prop={'size': 6})
        #plt.legend()

    if title is not None:
        plt.title(title)
    
    if x_axis_label is not None:
        plt.xlabel(x_axis_label)
    
    if y_axis_label is not None:
        plt.ylabel(y_axis_label)

    # plt.ylim([0, 1])
    # plt.xlim([0, 1])
    plt.margins(0.15, 0.25)
    plt.plot()
    

    os.makedirs("plots_thesis", exist_ok=True)
    output = f"plots_thesis/{experiment}_mixed_{x_axis.replace('@', '_')}_{y_axis.replace('@', '_')}"
    plt.savefig(output + ".pdf", dpi=300, bbox_inches='tight')

In [ ]:
experiments = {
    f"../{org_exp_dir}/rcv1x_100_plt": "RCV1x-2K",
    f"../{org_exp_dir}/eurlex_100_plt": "Eurlex-4K",
    f"../{org_exp_dir}/EURLex-4.3K_100_plt": "Eurlex-4.3K",
    f"../{org_exp_dir}/amazonCat_100_plt": "AmazonCat-13K",
    f"../{org_exp_dir}/amazonCat-14K_100_plt": "AmazonCat-14K",
    f"../{org_exp_dir}/wiki10_100_plt": "Wiki10-31K",
    #f"../{org_exp_dir}/deliciousLarge_100_plt": "DeliciousLarge-200K",
    f"../{org_exp_dir}/wikiLSHTC_100_plt": "WikiLSHTC-325K",
    f"../{org_exp_dir}/WikipediaLarge-500K_100_plt": "WikipediaLarge-500K",
    f"../{org_exp_dir}/amazon_100_plt": "Amazon-670K",


    f"../{syn_exp_dir}/rcv1x_100_plt": "Synthetic RCV1x-2K",
    f"../{syn_exp_dir}/eurlex_100_plt": "Synthetic Eurlex-4K",
    f"../{syn_exp_dir}/EURLex-4.3K_100_plt": "Synthetic Eurlex-4.3K",
    f"../{syn_exp_dir}/amazonCat_100_plt": "Synthetic AmazonCat-13K",
    f"../{syn_exp_dir}/amazonCat-14K_100_plt": "Synthetic AmazonCat-14K",
    f"../{syn_exp_dir}/wiki10_100_plt": "Synthetic Wiki10-31K",
    f"../{syn_exp_dir}/wikiLSHTC_100_plt": "Synthetic WikiLSHTC-325K",
    f"../{syn_exp_dir}/WikipediaLarge-500K_100_plt": "Synthetic WikipediaLarge-500K",
    f"../{syn_exp_dir}/amazon_100_plt": "Synthetic Amazon-670K",
}


METRICS = ["macro-f1", "macro-precision", "macro-recall"]
METRIC_LABELS = ["Macro-F1", "Macro-Precision", "Macro-Recall"]
METRICS = ["macro-f1", "macro-recall"]
METRIC_LABELS = ["Macro-F1", "Macro-Recall"]
METRIC_LABELS_SHORT = ["Macro-F1", "Macro-R"]

for METRIC, METRIC_LABEL, METRIC_LABEL_SHORT in zip(METRICS, METRIC_LABELS, METRIC_LABELS_SHORT):

    methods = {
        "optimal-instance-precision": "\\InfTopK",
        "optimal-instance-ps-precision": "\\InfPSK",
        "power-law-with-beta=0.25": "\\InfPowerK",
        "power-law-with-beta=0.5": "\\InfPowerK",
        "power-law-with-beta=0.75": "\\InfPowerK",
        "log": "\\InfLogK",
        "optimal-macro-recall": "\\InfMacR",
        "optimal-macro-balanced-accuracy": "\\InfMacBA",
    }
    for eps in EPSS:
        methods.update({
            f"block-coord-macro-precision-tol={TOL}-eps={eps}": f"\\InfBCAMacP$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-recall-tol={TOL}-eps={eps}": f"\\InfBCAMacR$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-balanced-accuracy-tol={TOL}-eps={eps}": f"\\InfBCAMacBA$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-f1-tol={TOL}-eps={eps}": f"\\InfBCAMacF$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"block-coord-macro-jaccard-score-tol={TOL}-eps={eps}": f"\\InfBCAMacJ$_{{\\epsilon={eps}}}$",
        })

    methods.update({
        f"block-coord-coverage-tol={TOL}": "\\InfBCACov",
    })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-precision-eps={eps}": f"\\InfFWMacP$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-recall-eps={eps}": f"\\InfFWMacR$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-balanced-accuracy-eps={eps}": f"\\InfFWMacBA$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-f1-eps={eps}": f"\\InfFWMacF$_{{\\epsilon={eps}}}$",
        })

    for eps in EPSS:
        methods.update({
            f"frank-wolfe-macro-jaccard-score-eps={eps}": f"\\InfFWMacJ$_{{\\epsilon={eps}}}$",
        })

    ALPHAS = [0.9, 0.7, 0.5, 0.3, 0.1, 0.05]
    for a in ALPHAS:
        #for eps in EPSS:
        methods.update({
            f"power-law-with-beta={a}": f"\\InfPowerK",
        })
        for eps in [1e-8, 1e-6, 1e-4]:
            methods.update({
                #f"block-coord-mixed-precision-macro-precision-alpha={a}-tol={TOL}-eps={eps}": f"\\InfBCAMacBA$_{{\\alpha={a}}}$",
                f"block-coord-mixed-precision-{METRIC}-alpha={a}-tol={TOL}-eps={eps}": f"\\InfBCAMacF$_{{\\alpha={a}}}$",
                #f"block-coord-mixed-precision-macro-recall-alpha={a}-tol={TOL}-eps={eps}": f"\\InfBCAMacR$_{{\\alpha={a}}}$",
            })
        for eps in EPSS:
            methods.update({
                f"frank-wolfe-mixed-precision-{METRIC}-alpha={a}-eps={eps}": f"\\InfFWMacF$_{{\\alpha={a}}}$",
            })


    plots = {
        "block-coord-mixed-precision-{METRIC}": {
            f"block-coord-{METRIC}": r"\InfBCA{$(1 \! - \! \lambda) \text{P}@k \! + \! \lambda \text{" + METRIC_LABEL_SHORT + r"}@k$}",
            #f"block-coord-mixed-precision-{METRIC}-alpha=0.95": f"",
            f"block-coord-mixed-precision-{METRIC}-alpha=0.9": f"",
            f"block-coord-mixed-precision-{METRIC}-alpha=0.7": f"",
            f"block-coord-mixed-precision-{METRIC}-alpha=0.5": f"",
            f"block-coord-mixed-precision-{METRIC}-alpha=0.3": f"",
            f"block-coord-mixed-precision-{METRIC}-alpha=0.1": f"",
            f"block-coord-mixed-precision-{METRIC}-alpha=0.05": f"",
            "optimal-instance-precision": "",
        },
        # "power-law": {
        #     f"optimal-macro-recall": f"",
        #     f"power-law-with-beta=0.9": f"",
        #     f"power-law-with-beta=0.7": f"",
        #     f"power-law-with-beta=0.5": f"",
        #     f"power-law-with-beta=0.3": f"",
        #     f"power-law-with-beta=0.1": f"",
        #     "optimal-instance-precision": "",
        # },
        "frank-wolfe-mixed-precision-{METRIC}": {
            f"frank-wolfe-{METRIC}": r"\InfFW{$(1 \! - \! \lambda) \text{P}@k \! + \! \lambda \text{" + METRIC_LABEL_SHORT + r"}@k$}",
            #f"frank-wolfe-mixed-precision-{METRIC}-alpha=0.95": f"",
            f"frank-wolfe-mixed-precision-{METRIC}-alpha=0.9": f"",
            f"frank-wolfe-mixed-precision-{METRIC}-alpha=0.7": f"",
            f"frank-wolfe-mixed-precision-{METRIC}-alpha=0.5": f"",
            f"frank-wolfe-mixed-precision-{METRIC}-alpha=0.3": f"",
            f"frank-wolfe-mixed-precision-{METRIC}-alpha=0.1": f"",
            f"frank-wolfe-mixed-precision-{METRIC}-alpha=0.05": f"",
            "optimal-instance-precision": "",
        },
        "optimal-instance-precision": "\\InfTopK",
        "optimal-instance-ps-precision": "\\InfPSK",
        "power-law-with-beta=0.25": "\\InfPowerK$_{\\beta=0.25}$",
        "power-law-with-beta=0.5": "\\InfPowerK$_{\\beta=0.5}$",
        #"power-law-with-beta=0.75": "\\InfPowerK$_{\\beta=0.75}$",
        "log": "\\InfLogK",
        #"optimal-macro-recall": "\\InfMacR",
    }



    for e, (experiment, experiment_label) in enumerate(experiments.items()):
        print(f"Processing {e}: {experiment} - {experiment_label}")
        results = []
        prev_method = ""
        for method, method_label in methods.items():
            _method = method.split("-eps")[0].split("-tol")[0]
            method_results = {
                "_method": _method,
                "method": method_label
            }
            if "midrule" in method:
                results.append(method_results)
                continue

            for seed in seeds:
                filename = f"{experiment}/{method}_k={top_k}_v={val_split}_s={seed}_results.json"
                if not os.path.exists(filename):
                    filename = filename.replace("org5", "org3")
                
                if not os.path.exists(filename):
                    filename = filename.replace("org3", "org2")
                    filename = filename.replace("syn3", "syn2")

                if os.path.exists(filename):
                    with open(filename, "r") as f:
                        result_file_data = json.load(f)
                    for metric, metric_label in metrics.items():
                        metric = f"{metric}@{top_k}"
                        if metric in result_file_data:
                            method_results.setdefault(metric, []).append(result_file_data[metric] * multiplier)
                else:
                    #print(f"File {filename} not found")
                    pass
            
            _method_results = {}
            for k, v in method_results.items():
                if isinstance(v, list) and isinstance(v[0], float):
                    method_results[k] = np.mean(v)
                    _method_results[k + "_std"] = np.std(v)
            method_results.update(_method_results)

            for metric in metrics.keys():
                if metric in method and metric in prev_method and _method == prev_method:
                    metric_at_k = f"{metric}@{top_k}"
                    new_val = method_results.get(metric_at_k, 0)
                    prev_val = results[-1].get(metric_at_k, 0)
                    if new_val > prev_val and isinstance(new_val, float):
                        results[-1] = method_results
                    break
            else:
                results.append(method_results)
            prev_method = _method

        for r in results:
            for k, v in r.items():
                if isinstance(v, float):
                    r[k] = np.round(v, 2)

        from pprint import pprint
        results = {r["_method"]: r for r in results}
        plot_results(experiment, results, plots, 
                    f"{METRIC}@{top_k}", 
                    f"instance-precision@{top_k}", 
                    x_axis_label=f"{METRIC_LABEL}$@{top_k}$", 
                    y_axis_label=f"Precision$@{top_k}$", 
                    title=experiment_label, 
                    legend=(METRIC == "macro-recall" and (e==0 or e == 5 or e == 9 or e == 14)), 
                    add_std=True,
                    on_plot_labels=False)